In [63]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [64]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("MachineLearning-Lecture01.pdf"),
    PyPDFLoader("MachineLearning-Lecture01.pdf"),
    PyPDFLoader("MachineLearning-Lecture02.pdf"),
    PyPDFLoader("MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [65]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [66]:
splits = text_splitter.split_documents(docs)

In [67]:
len(splits)

208

In [68]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [69]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [70]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [71]:
import numpy as np

In [72]:
np.dot(embedding1, embedding2)

np.float64(0.9631510802407719)

In [73]:
np.dot(embedding1, embedding3)

np.float64(0.7701699041109185)

In [74]:
np.dot(embedding2, embedding3)

np.float64(0.7590262031368057)

In [75]:
 ! pip install chromadb

Defaulting to user installation because normal site-packages is not writeable


In [81]:
from langchain.vectorstores import Chroma

In [85]:
persist_directory = 'docs/docs/chroma/'

In [86]:
!rm -rf ./docs/chroma  # remove old database files if any

In [87]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [88]:
print(vectordb._collection.count())

208


In [89]:
question = "is there an email i can ask for help"

In [90]:
docs = vectordb.similarity_search(question,k=4)

In [91]:
len(docs)

4

In [92]:
docs[0].page_content

"cs229-qa@cs.stanford.edu. This goes to an account that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework problems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thing that I think will help you to succeed and \ndo well in this class and even help you to enjoy this class more is if you form a study \ngroup.  \nSo start looking around where you're sitting now or at the end of class today, mingle a \nlittle bit and get to know your classmates. I strongly encourage you to form st

In [93]:
vectordb.persist()

In [94]:
question = "what did they say about matlab?"

In [95]:
docs = vectordb.similarity_search(question,k=5)

In [96]:
docs[0]

Document(metadata={'moddate': '2008-07-11T11:25:23-07:00', 'total_pages': 22, 'page_label': '9', 'author': '', 'creationdate': '2008-07-11T11:25:23-07:00', 'creator': 'PScript5.dll Version 5.2.2', 'title': '', 'page': 8, 'source': 'MachineLearning-Lecture01.pdf', 'producer': 'Acrobat Distiller 8.1.0 (Windows)'}, page_content='those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your own home computer or something if you \ndon\'t have a MATLAB license

In [97]:
docs[1]

Document(metadata={'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2008-07-11T11:25:23-07:00', 'page': 8, 'title': '', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'total_pages': 22, 'creationdate': '2008-07-11T11:25:23-07:00', 'source': 'MachineLearning-Lecture01.pdf', 'page_label': '9', 'author': ''}, page_content='those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your own home computer or something if you \ndon\'t have a MATLAB license

In [98]:
question = "what did they say about regression in the third lecture?"

In [99]:
docs = vectordb.similarity_search(question,k=5)

In [100]:
for doc in docs:
    print(doc.metadata)

{'moddate': '2008-07-11T11:25:03-07:00', 'page_label': '1', 'author': '', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:03-07:00', 'page': 0, 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'total_pages': 16, 'title': '', 'source': 'MachineLearning-Lecture03.pdf'}
{'creator': 'PScript5.dll Version 5.2.2', 'page_label': '3', 'moddate': '2008-07-11T11:25:05-07:00', 'total_pages': 18, 'title': '', 'creationdate': '2008-07-11T11:25:05-07:00', 'page': 2, 'author': '', 'source': 'MachineLearning-Lecture02.pdf', 'producer': 'Acrobat Distiller 8.1.0 (Windows)'}
{'creator': 'PScript5.dll Version 5.2.2', 'title': '', 'page_label': '18', 'author': '', 'moddate': '2008-07-11T11:25:05-07:00', 'source': 'MachineLearning-Lecture02.pdf', 'total_pages': 18, 'creationdate': '2008-07-11T11:25:05-07:00', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'page': 17}
{'author': '', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:23-07:00', 'producer': 

In [101]:
print(docs[4].page_content)

into his office and he said, "Oh, professor, professor, thank you so much for your 
machine learning class. I learned so much from it. There's this stuff that I learned in your 
class, and I now use every day. And it's helped me make lots of money, and here's a 
picture of my big house."  
So my friend was very excited. He said, "Wow. That's great. I'm glad to hear this 
machine learning stuff was actually useful. So what was it that you learned? Was it 
logistic regression? Was it the PCA? Was it the data networks? What was it that you 
learned that was so helpful?" And the student said, "Oh, it was the MATLAB."  
So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, 
and we'll actually have a short MATLAB tutorial in one of the discussion sections for 
those of you that don't know it.  
Okay. The very last piece of logistical thing is the discussion sections. So discussion 
sections will be taught by the TAs, and attendance at discussion sections is o

In [102]:
question = "what did they go for lunch?"

In [103]:
docs = vectordb.similarity_search(question,k=5)

In [107]:
print(docs[1].page_content)

middle of class, but because there won't be video you can safely sit there and make faces 
at me, and that won't show, okay?  
Let's see. I also handed out this — ther e were two handouts I hope most of you have, 
course information handout. So let me just say a few words about parts of these. On the 
third page, there's a section that says Online Resources.  
Oh, okay. Louder? Actually, could you turn up the volume? Testing. Is this better? 
Testing, testing. Okay, cool. Thanks.
